In [271]:
import os
import requests
import base64
from openai import OpenAI
import openai
from dotenv import load_dotenv
from PIL import Image


In [272]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
openai.api_key = os.getenv("API_KEY")
openai.api_type = os.getenv("API_TYPE")
openai.api_version = os.getenv("API_VERSION")
openai.azure_endpoint = os.getenv("AZURE_ENDPOINT")

In [273]:
MODEL="gpt-4o-mini"

In [274]:
# completion = openai.chat.completions.create(
#   model=MODEL,
#   temperature=0,
#   top_p=1,
#   max_tokens=2000,
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant that helps me with my math homework!"},
#     {"role": "user", "content": "Hello! Could you solve 20 x 5?"}
#   ]
# )
# print("Assistant: " + completion.choices[0].message.content)

In [275]:
meta_prompt = """
You are a helpful medical assistant tasked with analyzing the text from a medical prescription image and extracting relevant information. 
Your goal is to provide a clear and concise summary of the prescription based on the following key points.

- First predict if the image is of prescription or not, if not, return with output: "Not a Prescription". 
Otherwise, continue without saying it is prescription.

Key Information to Extract:
- Medical Advice (if present, otherwise skip).
- Medicine Name (if present, otherwise skip), and validate the name to ensure it’s a valid medicine. If not valid, correct the medicine name.
- Dosage (if present, otherwise skip), and explain the dosage instructions (e.g., twice daily, bd, od).
- Names of Tests (if present, otherwise skip), validate the names of the tests.
- Duration of Medication (if present, otherwise skip) – how long the medication should be followed.

Rules:
- For any medicine names, provide a brief explanation of the use of the medicine.
- Validate the medicine names against a list of recognized medicines to ensure accuracy.
- Validate the test names against a list of recognized medical tests to ensure accuracy.
- Interpret dosage shorthand:
   - "o---o" or two time o with a - in between means, twice daily
   - "bd" = one tablet after breakfast and one after dinner
   - "od" = one tablet daily
- Rx indicates a prescription, ensure it's handled appropriately.
- Do not repeat information twice within the response.

Output Format:
- The response should be a single paragraph without any use of bullet points, numbers, or special characters like ** or -.
- The response should include only the information that is present in the prescription and should be formatted in natural language.
"""

In [276]:
def get_chat_completion(language, image_file, model=MODEL, meta_prompt=meta_prompt):
    """
    This function sends a chat completion request to the OpenAI API with a given model,
    meta prompt, user message, and image in base64 format.

    Args:
        language (str): The text that the user sends (language).
        image_file (str): The file name of the image
        model (str): The model to be used (e.g., 'gpt-4o-mini').
        meta_prompt (str): The system-level prompt.

    Returns:
        str: The completion response from the API.
    """

    prompt = f"Return the responses in {language} language"
    image_path = os.path.join(os.getcwd(), image_file)
    encoded_image = base64.b64encode(open(image_file, 'rb').read()).decode('ascii')
    
    # Make the chat completion request
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": meta_prompt},
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{encoded_image}"}
                }
            ]}
        ],
        temperature=0,
        top_p=1,
        max_tokens=2000,
    )

   
    
    # Return the content of the completion
    result = response.choices[0].message.content
    print(response.choices[0].message.content)
    engine = pyttsx3.init()
    engine.say(result)
    engine.runAndWait()
    
    # Display the image using PIL
    img = Image.open(image_path)
    img.show()


In [284]:
get_chat_completion(language="Hindi", image_file="images/pres3.jpg")

इस प्रिस्क्रिप्शन में "Paicel 20" नामक दवा दी गई है, जो आमतौर पर दर्द निवारक के रूप में उपयोग होती है। इसे दिन में दो बार लेने की सलाह दी गई है। इसके अलावा "Apifex 160" नामक दवा भी है, जिसे दिन में एक बार लेने के लिए कहा गया है। अंत में "Linagliptin" नामक दवा का उल्लेख है, जिसे भी दिन में एक बार लेने की सलाह दी गई है।
